In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, isdir, join

In [68]:
x_data = []
y_data = []

In [69]:
mypath = "D:/py/MJ/data/" #這裡要改成實際放data的位置，請參考readme
files_list = listdir(mypath)

f_counter = 0
sp_counter = 0
sp_counter2 = 0

In [70]:
for f_name in files_list:
    f = open(mypath + f_name,'r')
    mjlog_str = f.read()
    f.close()
    mjlog_str = mjlog_str[mjlog_str.find('<UN'):] #hash後有可能剛好出現UN
    
    players = [0]*4
    players[0] = mjlog_str[mjlog_str.find('n0') + 4 : mjlog_str.find('n1') - 2]
    players[1] = mjlog_str[mjlog_str.find('n1') + 4 : mjlog_str.find('n2') - 2]
    players[2] = mjlog_str[mjlog_str.find('n2') + 4 : mjlog_str.find('n3') - 2]
    players[3] = mjlog_str[mjlog_str.find('n3') + 4 : mjlog_str.find('dan') - 2]
    
    if '%41%53%41%50%49%4E' not in players: #'%41%53%41%50%49%4E' == asapin
        print('get Asapin id error')
    else:
        for i in range(4):
            if players[i] == '%41%53%41%50%49%4E':
                master_player = i  

    counter = mjlog_str.count('INIT')

    mjlog_str_part = mjlog_str.split('INIT')
    mjlog_str_part = mjlog_str_part[1:]

    seed = [0,0,0]
    point = [250]*4

    for i in range(counter):

        dora_indicator = [-1] #最多翻到5張dora指示
        reach = [0,0,0,0]
        hand = []
        throw = [[] for j in range(4)] #[[]]*4 這樣子寫會爆炸.. 那樣裡面的[]都會視為同一個 list,不是我們想要的
        fuuro = [[] for j in range(4)]
        menzenchin = [0,0,0,0]

        seed_str = mjlog_str_part[i][mjlog_str_part[i].find('seed') + 6:mjlog_str_part[i].find('ten') - 2 ]
        seed_list = seed_str.split(',')

        seed[0] = int(seed_list[0])
        seed[1] = int(seed_list[1])
        seed[2] = int(seed_list[2]) #3,4是骰點, 不重要
        dora_indicator[0] = int(seed_list[5]) #dora指示
        oya = int(mjlog_str_part[i][mjlog_str_part[i].find('oya') + 5:mjlog_str_part[i].find('oya') + 6 ]) # oya要記得換成int

        start_playing_tag = mjlog_str_part[i].find('hai3')

        #讀手牌
        if master_player == 0:
            for k in mjlog_str_part[i][mjlog_str_part[i].find('hai0=') + 6:mjlog_str_part[i].find('hai1=') - 2 ].split(','):
                hand.append(int(k))
        elif master_player == 1:
            for k in mjlog_str_part[i][mjlog_str_part[i].find('hai1=') + 6:mjlog_str_part[i].find('hai2=') - 2 ].split(','):
                hand.append(int(k))
        elif master_player == 2:
            for k in mjlog_str_part[i][mjlog_str_part[i].find('hai2=') + 6:mjlog_str_part[i].find('hai3=') - 2 ].split(','):
                hand.append(int(k))
        elif master_player == 3:
            for k in mjlog_str_part[i][mjlog_str_part[i].find('hai3=') + 6:mjlog_str_part[i].find('\"', start_playing_tag + 8) ].split(','):
                hand.append(int(k))
        else:
            print('get master_player error')

        mj_str_start = mjlog_str_part[i][mjlog_str_part[i].find('/><', start_playing_tag) + 3:]

        mj_list_start = mj_str_start.split('/><')
        mj_list_start = mj_list_start[:-1] #最後面的 'mjloggm>' 用不到，所以去掉

        #開始摸牌與捨牌
        current = oya
        x_data 
        
        for j in range(len(mj_list_start)):

            if mj_list_start[j][0] in ['T','U','V','W']: #摸牌
                if mj_list_start[j][0:2] == 'UN': #有人連回來的時候..
                    continue
                if mj_list_start[j][0] == ['T','U','V','W'][master_player]: #asapin摸的才重要
                    hand.append(int(mj_list_start[j][1:]))
                    #寫入x_data 
                    if mj_list_start[j + 1][0] == 'A':##如果自摸就不用打牌了
                        sp_counter += 1
                        continue

                    x_datum = seed + (dora_indicator + [0]* (5 - len(dora_indicator)) ) + reach + (sorted(hand) + [0]* (14 - len(hand)) ) #最多翻到5張dora指示
                    for k in [0,1,2,3]:
                        temp = throw[k] + [0]*(22 - len(throw[k])) #假設最多會丟到22張牌，然後共四家，缺項補 0
                        x_datum += temp
                        
                    for k in [0,1,2,3]:
                        temp = fuuro[k] + [0]*(16 - len(fuuro[k])) #最多會有16張副露(aka 四槓子) 
                        x_datum += temp
                    x_datum += menzenchin    
                    x_datum += point

                    x_data.append(x_datum)
            
            elif mj_list_start[j][0:4] == 'DORA': #<DORA hai="72"/> #要翻DORA先判斷，避免和捨牌搞混
                dora_indicator.append(int(mj_list_start[j][10:-2]))

            elif mj_list_start[j][0] in ['D','E','F','G']: #捨牌
                if mj_list_start[j][0] == ['D','E','F','G'][master_player]:
                    y_data.append(int(mj_list_start[j][1:]))
                    
                    hand.remove(int(mj_list_start[j][1:]))
                throw[current].append(int(mj_list_start[j][1:]))
                current = (current + 1) % 4 #換下一家

            elif mj_list_start[j][0:2] == 'RE': #Reach
                reach[ int(mj_list_start[j][11:12]) ] += 1
            elif mj_list_start[j][0:2] == 'RY': #<RYUUKYOKU ba="1,0" sc="240,-15,259,15,142,-15,359,15" hai1="9,14,15,16,18,72,76,80,82,85,88,120,123" hai3="10,17,21,23"/>
                if mj_list_start[j-1][0] == ['T','U','V','W'][master_player]: #特殊流局沒有捨牌
                    x_data.pop()
                    sp_counter2 += 1
            elif mj_list_start[j][0] == 'N': #有人叫牌

                current = int(mj_list_start[j][7:8]) #<N who="1" m="43114"/>
                #print('current = ', current)
                m = int(mj_list_start[j][13:-2])
                m_str = '0'*( 16 - len(bin(m)[2:]) ) + bin(m)[2:] #統一長度為16，前面補0

                if m_str[13] == '1': #吃
                    chi = int(m_str[0:6],2)
                    chi_tag = chi % 3 # 0: 吃最小 1: 吃中間 2: 吃最大
                    chi_color = int(int(chi/3)/7) # 0萬1筒2條
                    chi_type = int(chi/3) % 7 #0:123 1:234 ... 6:789

                    fuuro[current].append(chi_color * 9 + chi_type)
                    fuuro[current].append(chi_color * 9 + chi_type + 1)
                    fuuro[current].append(chi_color * 9 + chi_type + 2)
                    
                    menzenchin[current] += 1
                    if current == master_player:
                        temp = [0,1,2]
                        temp.remove(chi_tag)
                        for k in temp:
                            hand.remove( (chi_color * 9 + chi_type + k)*4 + int(m_str[11-2*k:13-2*k],2) ) #0: 11~13 1: 9~11 2: 7~9

                        #寫入x_data
                        x_datum = seed + (dora_indicator + [0]* (5 - len(dora_indicator)) ) + reach + (sorted(hand) + [0]* (14 - len(hand)) ) #最多翻到5張dora指示
                        for k in [0,1,2,3]:
                            temp = throw[k] + [0]*(22 - len(throw[k])) #假設最多會丟到22張牌，然後共四家，缺項補 0
                            x_datum += temp
                        for k in [0,1,2,3]:
                            temp = fuuro[k] + [0]*(16 - len(fuuro[k])) #最多會有16張副露(aka 四槓子) 
                            x_datum += temp
                        x_datum += menzenchin    
                        x_datum += point

                        x_data.append(x_datum)
                        
                elif m_str[12] == '1': #碰
                    pon = int(m_str[0:7],2)
                    pon_tag = pon % 3
                    pon_which = int(pon/3) #萬筒條~字牌 [0-34]
                    temp = [0,1,2,3]
                    temp.remove(int(m_str[9:11],2))

                    for k in temp:
                        fuuro[current].append(pon_which * 4 + k)
                    menzenchin[current] += 1
                    if current == master_player:

                        temp.remove(temp[pon_tag])
                        for k in temp:
                            hand.remove(pon_which * 4 + k)

                        #寫入x_data
                        x_datum = seed + (dora_indicator + [0]* (5 - len(dora_indicator)) ) + reach + (sorted(hand) + [0]* (14 - len(hand)) ) #最多翻到5張dora指示
                        for k in [0,1,2,3]:
                            temp = throw[k] + [0]*(22 - len(throw[k])) #假設最多會丟到22張牌，然後共四家，缺項補 0
                            x_datum += temp
                        for k in [0,1,2,3]:
                            temp = fuuro[k] + [0]*(16 - len(fuuro[k])) #最多會有16張副露(aka 四槓子) 
                            x_datum += temp
                        x_datum += menzenchin    
                        x_datum += point

                        x_data.append(x_datum)
                        
                elif m_str[11] == '1': #加槓
                    plus_gan = int(m_str[0:7],2)

                    fuuro[current].append(int(plus_gan/3)*4 + int(m_str[9:11],2))
                    if current == master_player:
                        hand.remove(int(plus_gan/3)*4 + int(m_str[9:11],2))
                        x_data.pop() #加槓時摸牌沒有捨牌
                        sp_counter2 += 1
                else: #大明槓或暗槓 也就是 m_str[11:14] == '000'
                    gan = int(m_str[0:8],2) ##此處有8位, gan: [0-135]

                    if int(m_str) & 3 == 0: #暗槓
                        for k in range(4): 
                            fuuro[current].append(int(gan/4) * 4 + k)
                        if current == master_player:
                            for k in range(4):
                                hand.remove(int(gan/4) * 4 + k)
                            x_data.pop() #暗槓時摸牌沒有捨牌
                            sp_counter2 += 1
                    else: #大明槓  
                        for k in range(4): 
                            fuuro[current].append(int(gan/4) * 4 + k)
                        if current == master_player:
                            for k in range(4):
                                if int(gan/4) * 4 + k in hand:
                                    hand.remove(int(gan/4) * 4 + k)
                        menzenchin[current] += 1    
                        
                current = (current + 1) % 4 #換下一家    

            elif mj_list_start[j][0] == 'A': #Agari
                pass
            
            elif mj_list_start[j][0] == 'B': #<BYE who="3"/>
                pass
            else :
                print('出錯惹')
                print(mj_list_start[j])
    if f_counter % 512 == 0:
        print('已完成.. ', f_counter/49,'%')
    f_counter += 1
    if len(x_data) != len(y_data):
        break
print('done!')

已完成..  0.0 %
已完成..  10.448979591836734 %
已完成..  20.897959183673468 %
已完成..  31.346938775510203 %
已完成..  41.795918367346935 %
已完成..  52.244897959183675 %
已完成..  62.69387755102041 %
已完成..  73.14285714285714 %
已完成..  83.59183673469387 %
已完成..  94.04081632653062 %
done!


In [71]:
len(x_data)

441097

In [72]:
len(y_data)

441097

In [73]:
len(x_data[1235])

186

In [21]:
#原本想輸出data檔的，不過發現很蠢，因為還要再處理資料，而且這樣的輸出檔案太大了多達100+MB..就放棄，直接作惹
#f = open('data.txt', 'w', encoding = 'UTF-8')

In [22]:
#for i in range(len(x_data)):
#    for j in range(len(x_data[i])):
#        print(x_data[i][j],file = f, end =' ')
#    print(y_data[i],file = f)

In [23]:
#f.close()

In [24]:
env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [39]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, concatenate
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import SGD, Adam

In [74]:
x_train  = np.array(x_data[0:300000])
y_train = np.array(y_data[0:300000])
x_test = np.array(x_data[300000:])
y_test = np.array(y_data[300000:])

y_train = (y_train/4).astype(int)
y_test = (y_test/4).astype(int)

y_train = np_utils.to_categorical(y_train, 34)
y_test = np_utils.to_categorical(y_test, 34)

In [78]:
model = Sequential()
model.add(Dense(256, input_dim=186)) #第一層256個神經原
model.add(Activation('relu'))
model.add(Dense(128)) #第二層128個神經原
model.add(Activation('relu'))
model.add(Dense(56)) #第三層72個神經原
model.add(Activation('relu'))
model.add(Dense(34)) #輸出層
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer=Adam(),
                metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 256)               47872     
_________________________________________________________________
activation_27 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               32896     
_________________________________________________________________
activation_28 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 56)                7224      
_________________________________________________________________
activation_29 (Activation)   (None, 56)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 34)                1938      
__________

In [77]:
model_his = model.fit(x_train, y_train, 
            batch_size=80, 
            epochs=8, 
            verbose=1, 
            validation_data=(x_test, y_test))

Train on 300000 samples, validate on 141097 samples
Epoch 1/8
300000/300000 [==============================] - 31s 104us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 2/8
300000/300000 [==============================] - 29s 96us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 3/8
300000/300000 [==============================] - 29s 97us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 4/8
300000/300000 [==============================] - 28s 93us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 5/8
300000/300000 [==============================] - 31s 105us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 6/8
300000/300000 [==============================] - 29s 96us/step - loss: 0.1301 - acc: 0.9706 - val_loss: 0.1302 - val_acc: 0.9706
Epoch 7/8
300000/300000 [==============================] - 27s 92us/step - loss: 0.1301 - acc: 0.9706 - val_loss

## 最後成果有達到97%的預測準確率，而且測資也有相應的高準確率
### 在這次專案中，我認為花費最多時間處理的就是在處理資料，由於原始檔案是類似html檔，因此在處理的時候不太容易，所幸最後成果不差，希望之後有時間可以繼續完整這個專案，以達到真正的日麻AI
